In [1]:
import cv2
import os

def crop_burn_region(image_path, label_path, output_folder):
    # Read image and get dimensions
    image = cv2.imread(image_path)
    if image is None:
        print("Image not found:", image_path)
        return
    h, w = image.shape[:2]

    # Open label file and read bounding boxes
    with open(label_path, 'r') as file:
        lines = file.readlines()
    
    # Process each bounding box
    for i, line in enumerate(lines):
        parts = line.strip().split()
        if len(parts) != 5:
            continue  # Skip malformed lines
        # The first value is the burn degree (class), followed by normalized coordinates
        burn_degree, x_center_norm, y_center_norm, width_norm, height_norm = map(float, parts)
        
        # Convert normalized values to pixel coordinates
        x_center = x_center_norm * w
        y_center = y_center_norm * h
        box_width = width_norm * w
        box_height = height_norm * h
        
        # Calculate bounding box corners
        x_min = int(max(x_center - box_width / 2, 0))
        y_min = int(max(y_center - box_height / 2, 0))
        x_max = int(min(x_center + box_width / 2, w))
        y_max = int(min(y_center + box_height / 2, h))
        
        # Crop the burn region
        crop = image[y_min:y_max, x_min:x_max]
        
        # Create output filename including the burn degree
        base_name = os.path.basename(image_path)
        name, ext = os.path.splitext(base_name)
        degree = int(burn_degree)  # Convert burn degree to an integer
        output_filename = f"{name}_degree_{degree}_crop_{i}{ext}"
        output_path = os.path.join(output_folder, output_filename)
        
        # Save the cropped image
        cv2.imwrite(output_path, crop)
        print("Saved cropped image:", output_path)

# Example usage:
image_file = "path/to/your/image.jpg"
label_file = "path/to/your/image.txt"
output_dir = "path/to/output/folder"
#crop_burn_region(image_file, label_file, output_dir)
